In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "HIV_Resistance"
cohort = "GSE46599"

# Input paths
in_trait_dir = "../../input/GEO/HIV_Resistance"
in_cohort_dir = "../../input/GEO/HIV_Resistance/GSE46599"

# Output paths
out_data_file = "../../output/preprocess/HIV_Resistance/GSE46599.csv"
out_gene_data_file = "../../output/preprocess/HIV_Resistance/gene_data/GSE46599.csv"
out_clinical_data_file = "../../output/preprocess/HIV_Resistance/clinical_data/GSE46599.csv"
json_path = "../../output/preprocess/HIV_Resistance/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide analysis of interferon-stimulated genes in primary cells and immortalized cell lines"
!Series_summary	"Analysis of interferon-stimulated genes (ISGs) in various primary cells and immortalized cell lines, following type 1 interferon (IFN) treatment. Some cell types become resistant to HIV-1 infection following type 1 interferon treatment (such as macrophages, THP-1, PMA-THP-1, U87-MG cells and to a lesser extent, primary CD4+ T cells) while others either become only partially resistant (e.g., HT1080, PMA-U937) or remain permissive (e.g., CEM, CEM-SS, Jurkat T cell lines and U937); for more information see (Goujon and Malim, Journal of Virology 2010) and (Goujon and Schaller et al., Retrovirology 2013). We hypothesized that the anti-HIV-1 ISGs are differentially induced and expressed in restrictive cells compared to permissive cells and performed a whole genome analysis following type 1 IFN treatment in cell types exhibiting different HI

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data for interferon-stimulated genes
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For HIV_Resistance: identified in key 4 "resistance to hiv-1 following ifn treatment"
trait_row = 4

# Age is not available in the sample characteristics
age_row = None

# Gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert HIV resistance trait to binary:
    1 = resistant
    0 = not resistant (permissive or partially resistant)
    None = untreated or unknown
    """
    if value is None:
        return None
    
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "resistant":
        return 1
    elif value in ["permissive", "partially resistant"]:
        return 0
    elif value == "untreated":
        return None
    else:
        return None

# Since age and gender data are not available, define basic converter functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Conduct initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    try:
        # Try loading from standard location first
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"), index_col=0)
    except FileNotFoundError:
        # If not found, try to find or construct the data from available sources
        # Get all files in the cohort directory
        files = os.listdir(in_cohort_dir)
        
        # Look for any files that might contain clinical information
        clinical_files = [f for f in files if "characteristic" in f.lower() or "clinical" in f.lower() or "phenotype" in f.lower()]
        
        if clinical_files:
            # Try to load the first matching file
            clinical_data = pd.read_csv(os.path.join(in_cohort_dir, clinical_files[0]), index_col=0)
        else:
            # If no suitable file is found, create a placeholder DataFrame
            # with the structure expected by geo_select_clinical_features
            # This should have one column per sample and rows for each characteristic
            print("No clinical data file found. Creating minimal DataFrame for trait information only.")
            
            # Create a simple DataFrame with just the trait information
            # Minimal structure with trait row and sample columns
            sample_data = {
                "sample1": "resistance to hiv-1 following ifn treatment: resistant",
                "sample2": "resistance to hiv-1 following ifn treatment: permissive",
                "sample3": "resistance to hiv-1 following ifn treatment: partially resistant",
                "sample4": "resistance to hiv-1 following ifn treatment: untreated"
            }
            clinical_data = pd.DataFrame({
                trait_row: [sample_data[s] for s in sample_data]
            }, index=sample_data.keys())
            
            # We're only creating a placeholder to demonstrate the structure
            print("Warning: Using placeholder clinical data. Real analysis requires actual sample data.")
    
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error during clinical feature extraction: {str(e)}")
        print("Skipping clinical feature extraction due to data structure issues.")
        print("Note: This cohort has trait information but couldn't be processed in standard format.")


No clinical data file found. Creating minimal DataFrame for trait information only.
Error during clinical feature extraction: Length mismatch: Expected axis has 0 elements, new values have 1 elements
Skipping clinical feature extraction due to data structure issues.
Note: This cohort has trait information but couldn't be processed in standard format.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 69
Header line: "ID_REF"	"GSM1133032"	"GSM1133033"	"GSM1133034"	"GSM1133035"	"GSM1133036"	"GSM1133037"	"GSM1133038"	"GSM1133039"	"GSM1133040"	"GSM1133041"	"GSM1133042"	"GSM1133043"	"GSM1133044"	"GSM1133045"	"GSM1133046"	"GSM1133047"	"GSM1133048"	"GSM1133049"	"GSM1133050"	"GSM1133051"	"GSM1133052"	"GSM1133053"	"GSM1133054"	"GSM1133055"	"GSM1133056"	"GSM1133057"	"GSM1133058"	"GSM1133059"	"GSM1133060"	"GSM1133061"	"GSM1133062"	"GSM1133063"	"GSM1133064"	"GSM1133065"	"GSM1133066"	"GSM1133067"	"GSM1133068"	"GSM1133069"	"GSM1133070"	"GSM1133071"	"GSM1133072"	"GSM1133073"	"GSM1133074"	"GSM1133075"	"GSM1133076"	"GSM1133077"	"GSM1133078"	"GSM1133079"
First data line: "ILMN_1343291"	15.19802012	15.11333259	15.20127512	15.1206348	15.20127512	15.21882761	15.22919968	15.12717254	15.19802012	15.21882761	15.04530416	15.20127512	15.0085115	15.18504016	15.04367475	15.26469393	15.1436849	15.15197724	15.23978664	15.11794106	15.04875307	15.33007355	15.01588861	14.96164359	15.17472

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228',
       'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651236',
       'ILMN_1651238', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259',
       'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278',
       'ILMN_1651281', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers from the previous step output

# The identifiers starting with "ILMN_" indicate Illumina probe IDs, not human gene symbols
# These are microarray probe identifiers from Illumina platform that need to be mapped to gene symbols
# For example, ILMN_1343291 is an Illumina probe ID, not a standard human gene symbol

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:


Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE46599
Line 6: !Series_title = Genome-wide analysis of interferon-stimulated genes in primary cells and immortalized cell lines
Line 7: !Series_geo_accession = GSE46599
Line 8: !Series_status = Public on Sep 13 2013
Line 9: !Series_submission_date = May 02 2013
Line 10: !Series_last_update_date = Dec 01 2022
Line 11: !Series_pubmed_id = 24048477
Line 12: !Series_pubmed_id = 36161446
Line 13: !Series_summary = Analysis of interferon-stimulated genes (ISGs) in various primary cells and immortalized cell lines, following type 1 interferon (IFN) treatment. Some cell types become resistant to HIV-1 infection following type 1 interferon treatment (such as macrophages, THP-1, PMA-THP-1, U87-MG cells and to a lesser extent, primary CD4+ T cell


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# Step 1: Identify which columns in gene annotation contain probe IDs and gene symbols
# From the preview, we can see:
# - "ID" column contains Illumina probe IDs like "ILMN_1343048"
# - "Symbol" column contains gene symbols like "phage_lambda_genome", "thrB"

# Step 2: Extract mapping between probe IDs and gene symbols
# Create a mapping dataframe with only the relevant columns
mapping_df = gene_annotation[['ID', 'Symbol']].copy()
mapping_df = mapping_df.dropna(subset=['Symbol'])  # Remove rows without gene symbols
# Rename 'Symbol' to 'Gene' to match the expected column name in apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'Symbol': 'Gene'})
print(f"Number of mappable probes: {len(mapping_df)}")
print("Preview of mapping dataframe:")
print(mapping_df.head())

# Step 3: Apply gene mapping to convert probe-level data to gene expression data
try:
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Check the result
    print(f"Resulting gene expression data shape: {gene_data.shape}")
    print("Preview of gene expression data:")
    print(gene_data.head())
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene mapping: {str(e)}")
    print("Gene mapping failed. Check the structure of your mapping dataframe and gene expression data.")


Number of mappable probes: 44837
Preview of mapping dataframe:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Resulting gene expression data shape: (19428, 48)
Preview of gene expression data:
       GSM1133032  GSM1133033  GSM1133034  GSM1133035  GSM1133036  GSM1133037  \
Gene                                                                            
A1BG     5.662013    5.323517    6.473583    5.874020    5.736427    5.818472   
A1CF    16.273260   17.492853   15.699672   15.778524   15.830556   15.665282   
A26C3   16.724024   15.905628   16.344077   16.558834   15.962810   16.514401   
A2BP1   20.912821   20.809613   21.722912   21.300184   20.866304   21.050868   
A2LD1    8.259530    8.040645    8.182764    7.862223    8.172796    7.544583   

       GSM1133038  GSM1133039  GSM113304

Gene expression data saved to ../../output/preprocess/HIV_Resistance/gene_data/GSE46599.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Since we didn't successfully save clinical data in previous steps, let's extract it again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_df = get_background_and_clinical_data(matrix_file)

# Define conversion functions based on Step 2's correct implementation
def convert_trait(value):
    """
    Convert HIV resistance trait to binary:
    1 = resistant
    0 = not resistant (permissive or partially resistant)
    None = untreated or unknown
    """
    if value is None:
        return None
    
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "resistant":
        return 1
    elif value in ["permissive", "partially resistant"]:
        return 0
    elif value == "untreated":
        return None
    else:
        return None

def convert_age(value):
    return None

def convert_gender(value):
    return None

# Extract clinical features with the correct row indices from Step 2
trait_row = 4  # Correct row for HIV resistance from Step 2
age_row = None  # From Step 2
gender_row = None  # From Step 2

# Extract clinical features
try:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print("Selected clinical features:")
    print(selected_clinical_df)

    # Save clinical data for future reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data columns preview:")
    print(list(linked_data.columns[:10]))  # Show first 10 column names

    # Check if we have any valid trait values before proceeding
    if linked_data[trait].notna().sum() == 0:
        print("Warning: No valid trait values found in the linked data.")
        print("This may indicate an issue with the trait extraction or mapping.")
        is_gene_available = len(normalized_gene_data) > 0
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=is_gene_available, 
            is_trait_available=False,  # No usable trait data
            is_biased=True,  # Mark as biased as we have no trait data
            df=linked_data,
            note="No valid trait values were found in the clinical data after extraction."
        )
        print("Data was determined to be unusable due to missing trait values and was not saved")
    else:
        # 4. Handle missing values
        print("\nMissing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        if gene_cols:
            missing_genes_pct = linked_data[gene_cols].isna().mean()
            genes_with_high_missing = sum(missing_genes_pct > 0.2)
            print(f"  Genes with >20% missing: {genes_with_high_missing}")
            
            if len(linked_data) > 0:  # Ensure we have samples before checking
                missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
                samples_with_high_missing = sum(missing_per_sample > 0.05)
                print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

        # Handle missing values
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")

        # Only proceed with further evaluation if we have data left
        if len(cleaned_data) > 0:
            # 5. Evaluate bias in trait and demographic features
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

            # 6. Final validation and save
            note = "Dataset contains gene expression data from HIV resistance studies. "
            if 'Age' in cleaned_data.columns:
                note += "Age data is available. "
            if 'Gender' in cleaned_data.columns:
                note += "Gender data is available. "

            is_gene_available = len(normalized_gene_data) > 0
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=is_gene_available, 
                is_trait_available=True, 
                is_biased=trait_biased, 
                df=cleaned_data,
                note=note
            )

            # 7. Save if usable
            if is_usable and len(cleaned_data) > 0:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                cleaned_data.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Data was determined to be unusable or empty and was not saved")
        else:
            print("After handling missing values, no samples remained. Data is unusable.")
            is_gene_available = len(normalized_gene_data) > 0
            validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=is_gene_available, 
                is_trait_available=True,
                is_biased=True,  # Mark as biased since no samples survived processing
                df=pd.DataFrame(),  # Empty dataframe
                note="No samples remained after handling missing values."
            )
except Exception as e:
    print(f"Error during processing: {str(e)}")
    # Record failure information
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=len(normalized_gene_data) > 0, 
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Processing failed with error: {str(e)}"
    )
    print("Data processing failed and was not saved")

Gene data shape after normalization: (18626, 48)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Gene data saved to ../../output/preprocess/HIV_Resistance/gene_data/GSE46599.csv
Selected clinical features:
                GSM1133032  GSM1133033  GSM1133034  GSM1133035  GSM1133036  \
HIV_Resistance         0.0         0.0         NaN         NaN         0.0   

                GSM1133037  GSM1133038  GSM1133039  GSM1133040  GSM1133041  \
HIV_Resistance         0.0         NaN         NaN         0.0         0.0   

                ...  GSM1133070  GSM1133071  GSM1133072  GSM1133073  \
HIV_Resistance  ...         NaN         NaN         1.0         1.0   

                GSM1133074  GSM1133075  GSM1133076  GSM1133077  GSM1133078  \
HIV_Resistance         NaN         NaN         0.0         0.0         NaN   

                GSM1133079  
HIV_Resistance         NaN  

[1 rows x 48 columns]
Clinical data saved to ../../output/preprocess/HIV_Resistance/clinical_data/GSE46599.csv
Linked data shape: (48, 18627)
Linked data columns preview:
['HIV_Resistance', 'A1BG', 'A1BG-AS1', 'A1CF', 

Data shape after handling missing values: (24, 18627)
For the feature 'HIV_Resistance', the least common label is '0.0' with 12 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'HIV_Resistance' in this dataset is fine.



Linked data saved to ../../output/preprocess/HIV_Resistance/GSE46599.csv
